In [20]:
from transformers import AutoTokenizer, AutoModelForTokenClassification,BitsAndBytesConfig
from peft import PeftModel
import torch
import numpy as np

In [21]:
# Step 1: Load base model and tokenizer
base_model = "mistralai/Mistral-7B-Instruct-v0.1"
adapter_model = "Aghori/mistral-medical-ner-qlora"

In [22]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    use_fast=True  # Optional, but usually better performance
)
# Fix: set pad_token
tokenizer.pad_token = tokenizer.eos_token

In [23]:
# Configure quantization for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16  # fallback to float16 for broader support
)

In [24]:
id2label = {
    0: "O",
    1: "B-Disease",
    2: "I-Disease",
    3: "B-Medication",
    4: "I-Medication",
    5: "B-Person",
    6: "I-Person",
    7: "B-Location",
    8: "I-Location"
}


In [26]:
# Load model with quantization (this makes it QLoRA)
model = AutoModelForTokenClassification.from_pretrained(
    base_model,
    quantization_config=bnb_config,  # This enables quantization
    device_map="auto",
    trust_remote_code=True,
    num_labels=len(id2label )  # Set your number of NER labels
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForTokenClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-Instruct-v0.1 and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
model = PeftModel.from_pretrained(model, adapter_model)

In [28]:
model.eval()

PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): MistralForTokenClassification(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
 

In [32]:
# Make sure the model is on the correct device (CUDA if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): MistralForTokenClassification(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
 

In [36]:
# Step 2: Tokenize the input text
def preprocess_text(text):
    # Tokenize and encode the input text, ensuring proper padding
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    return inputs

In [61]:
# Step 3: Define a function to run inference
def run_inference(text):
    inputs = preprocess_text(text)
    
    # Move input tensors to the correct device (GPU/CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # Run the model (make sure it's on the correct device)
    with torch.no_grad():
        outputs = model(**inputs)

    # Step 4: Post-process the outputs
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    # Decode the token-level predictions back to labels
    predicted_labels = predictions[0].cpu().numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].cpu().numpy())
    
    # Return the tokens along with their predicted labels
    results = []
    for token, label in zip(tokens, predicted_labels):
        label_name = id2label.get(label, "UNKNOWN")  # Map to actual label
        results.append((token, label_name))

    return results

In [96]:
def response_process(text):
    predictions = run_inference(text)
    for item in predictions:
        token = item[0].replace("▁", "")
        if token in text:
            label = item[1].split("-")[-1]
            print(f"Token: {token}, Predicted Label: {label}")

In [97]:
# Example usage:
text = "The patient was prescribed 20 mg of aspirin for the pain."
response_process(text)



Token: The, Predicted Label: O
Token: patient, Predicted Label: O
Token: was, Predicted Label: O
Token: presc, Predicted Label: O
Token: ribed, Predicted Label: O
Token: , Predicted Label: Medication
Token: 2, Predicted Label: Medication
Token: 0, Predicted Label: Medication
Token: mg, Predicted Label: Medication
Token: of, Predicted Label: Medication
Token: aspir, Predicted Label: Medication
Token: in, Predicted Label: Medication
Token: for, Predicted Label: O
Token: the, Predicted Label: O
Token: pain, Predicted Label: Disease
Token: ., Predicted Label: O
